In [1]:
"""
    Instruções:
        1- Responda as perguntas listadas no README.md;
        2- Faça as análises que julgar necessárias e deixe os resultados de suas análises plotadas no notebook.
"""

'\n    Instruções:\n        1- Responda as perguntas listadas no README.md;\n        2- Faça as análises que julgar necessárias e deixe os resultados de suas análises plotadas no notebook.\n'